# **CS 6120: Natural Language Processing - Prof. Ahmad Uzair** 

### **Assignment 2: n-gram Language Models and Hierarchical Clustering **

### **Total points: 100**

In this assignment, You will be learning character level language models and hierarchical Clustering by implementing it. 

## <CENTER>PART-A 

### OBJECTIVE : 

Your task is to train n-gram language models. [Ref SLP Chapter 3]

- Task 1: You will train unigram, bigram, and trigram models on given training files. Then you will score on given test files for unigram, bigram, and trigram. you will generate sentences from the trained model and compute perplexity.
- Task 2: You will create training data for n > 3. and Repeat the above task from training model.
<h6>Part-A = (55 Points) </h6>

In [101]:
'''
Your imports go here
You are encouraged to implement your own functions and not use from library.
'''
import sys
from collections import Counter
import numpy as np

In [102]:
# constants to define pseudo-word tokens
# access via UNK, for instance
# for this assignemnt we will follow <s> tag for beginning of sentence and
# </s> for end of senetence as suggested in SLP Book. Check sample training files for reference.
UNK = "<UNK>"
SENT_BEGIN = "<s>"
SENT_END = "</s>"

We need to initialise global variables for model

In [103]:

"""Initializes Parameters:
  n_gram (int): the n-gram order.
  is_laplace_smoothing (bool): whether or not to use Laplace smoothing
  threshold: words with frequency  below threshold will be converted to token
"""
# Initializing different object attributes
n_gram = 3
is_laplace_smoothing = True
vocab = [] 
n_gram_counts = {}
n_minus_1_gram_counts = None
threshold = 1

### TASK - 1  = 20 points :
Implement training function

In [104]:
def make_ngrams(tokens: list, n: int) -> list:
    """Creates n-grams for the given token sequence.
    Args:
    tokens (list): a list of tokens as strings
    n (int): the length of n-grams to create

    Returns:
    list: list of tuples of strings, each tuple being one of the individual n-grams
    """
    n_grams = [tuple(tokens[i: i+n]) for i in range(len(tokens) - n + 1)]
    ## Your code here 
    return n_grams

In [105]:
def train(training_file_path):
    """Trains the language model on the given data. Input file that
    has tokens that are white-space separated, has one sentence per line, and
    that the sentences begin with <s> and end with </s>
    Parameters:
      training_file_path (str): the location of the training data to read

    Returns:
    N Gram Counts, Vocab, N Minus 1 Gram Counts
    """
    with open(training_file_path, 'r') as fh:
      content = fh.read().split() # Read and split data to get list of words
    
    # Get the count of each word
    word_count = {}
    for word in content:
      # if word != SENT_BEGIN and word != SENT_END:
      word_count[word] = word_count.get(word, 0) + 1

    # Replace the words with <UNK> if count is < threshold(=1)
    content = [a if word_count.get(a, 0) >= threshold else UNK for a in content]
    # make use of make_n_grams function
    n_gram_tokens = make_ngrams(content, n_gram)
    n_gram_counts = {}
    for token in n_gram_tokens:
      n_gram_counts[token] = n_gram_counts.get(token, 0) + 1
    # Get the training data vocabulary
    vocab = set(content)
    # For n>1 grams compute n-1 gram counts to compute probability
    if n_gram > 1:
      n_minus_1_gram_counts = {}
      n_minus_1_gram_tokens = make_ngrams(content, n_gram - 1)
      for token in n_minus_1_gram_tokens:
        n_minus_1_gram_counts[token] = n_minus_1_gram_counts.get(token, 0) + 1
    return n_gram_counts, vocab, n_minus_1_gram_counts

Output your Trained Data Parameters:

In [ ]:
n_gram_counts, vocab, n_minus_1_gram_counts = train("train_data/berp-training-tri.txt")
print(n_gram_counts)
print(vocab)

### TASK - 2  = 15 points :
Implement Score function that will take input sentence and output probability of given string representing a single sentence.

In [98]:
def get_gram_log_prob(gram_counts, gram_token, total_value, V):
  return np.log((gram_counts.get(gram_token, 0) + 1) / (total_value + V))

def score(sentence):
    """Calculates the probability score for a given string representing a single sentence.
    Parameters:
      sentence (str): a sentence with tokens separated by whitespace to calculate the score of
      
    Returns:
      float: the probability value of the given string for this model
    """
    # Split the input sentence and replace out of vocabulary tokens with <UNK>     
    content = sentence.split()
    tokens = [a if a in vocab else UNK for a in content]
    # Calculate probability for each word and multiply(or take log and sum) them to get the sentence probability
    n_gram_tokens = make_ngrams(tokens, n_gram)
    n_minus_1_gram_tokens = make_ngrams(tokens, n_gram - 1)
    
    # assert len(n_gram_tokens) == len(n_minus_1_gram_tokens) - 1

    V = len(vocab)
    n_gram_total_occurence = sum(n_gram_counts.values())
    n_minus_1_gram_total_occurence = sum(n_minus_1_gram_counts.values())
    # log_prob = get_gram_log_prob(n_minus_1_gram_counts, n_minus_1_gram_tokens[0], n_minus_1_gram_total_occurence, V)
    log_prob = 0.
    
    for i in range(len(n_gram_tokens)):
      log_prob += get_gram_log_prob(n_gram_counts, n_gram_tokens[i], n_gram_total_occurence, V) - get_gram_log_prob(n_minus_1_gram_counts, n_minus_1_gram_tokens[i+1], n_minus_1_gram_total_occurence, V)
    return log_prob

In [99]:
with open("test_data/hw2-test-tri.txt", 'r') as fh:
    test_content = fh.read().split("\n")
num_sentences_1 = len(test_content)
ten_sentences_1 = test_content[:10]
print("# of test sentences: ", num_sentences_1)
probablities = []

# of test sentences:  100


In [100]:
# print probabilities/score of sentences in test content
for sentence in test_content:
  probablities.append(score(sentence))
probablities = np.array(probablities)
mean = np.mean(probablities)
std_dev = np.std(probablities)
print(probablities)
print(mean)
print(std_dev)

[ -9.62919467  -9.08664378 -14.43860946 -10.26403105 -15.84579328
  -9.40635558 -11.89418833 -13.67019696 -14.51702791  -8.81829239
  -8.81827842  -9.9783271   -8.58901126  -6.27453127 -12.72893927
  -6.83462795  -7.43199803  -7.6337237  -12.32101726 -11.95729738
 -24.72462736 -18.75078274 -17.5781891  -15.99296055 -11.98249451
 -11.2846587   -7.15850146 -10.04885925 -15.69625298 -14.12313717
  -9.92045055 -19.64853378 -11.42118572 -13.51564497 -15.85570058
 -15.04749102  -8.25910777  -8.41091076 -11.36592056 -10.35326334
 -12.12451791 -11.05704468 -10.74928434 -13.57856054 -12.04615136
 -10.18135735  -9.04140801 -11.16963971 -12.92269157 -15.74217472
 -12.40171357 -12.19596036  -5.4906129   -4.03613369  -4.03613369
 -12.5023427  -12.64922474  -8.12514521  -6.33557972 -16.0424991
  -3.44206078 -11.9106039  -17.440516    -8.41278538 -18.46283213
 -13.55822026  -8.81827842  -6.73885085  -8.12514521 -18.57026706
 -17.83387865 -15.47998286  -9.28665649  -9.17492544 -17.58147983
  -8.818264

### TASK - 3  = 10 points :
Generate sentence from the above trained model

In [150]:
import random

def generate_sentence():
  """Generates a single sentence from a trained language model using the Shannon technique.
    
  Returns:
    str: the generated sentence
  """
  # Start with <s> and randomly generate words until we encounter sentence end
  # Append sentence begin markers for n>2
  # Keep track of previous word for stop condition
  sentence = ['<s>']
  prev_word = '<s>'
  if n_gram > 1:
    if n_gram > 2:
      for _ in range(n_gram - 2):
        sentence.append('<s>')
    prev_token = ['<s>' for _ in range(n_gram-1)]
    while prev_word != "</s>":
      # Construct the (n-1) gram so far
      # Get the counts of all available choices based on n-1 gram
      # Convert the counts into probability for random.choice() function
      # If <s> is generated, ignore and generate another word'
      candidate_tokens = []
      candidate_tokens_occurance = []
      for token in n_gram_counts:
        if list(token[:n_gram-1]) == prev_token:
          candidate_tokens.append(token)
          candidate_tokens_occurance.append(n_gram_counts[token])
      picked_token = random.choices(candidate_tokens, candidate_tokens_occurance, k=1)[0]
      picked_word = picked_token[-1]
      if picked_word != '<s>':
        sentence.append(picked_word)
        prev_word = picked_word
        prev_token = prev_token[1:]
        prev_token.append(picked_word)

  else:
    # In case of unigram model, n-1 gram is just the previous word and possible choice is whole vocabulary
    while prev_word != "</s>":
      # Convert the counts into probability for random.choice() function
      # If <s> is generated, ignore and generate another word
      picked_word = random.choice(vocab)
      if picked_word != '<s>':
        sentence.append(picked_word)
        prev_word = picked_word

  # Append sentence end markers for n>2
  if n_gram > 2:
    for _ in range(n_gram - 2):
      sentence.append('</s>')
  return " ".join(sentence)

In [148]:
def generate(n):
    """Generates n sentences from a trained language model using the Shannon technique.
    Parameters:
      n (int): the number of sentences to generate
      
    Returns:
      list: a list containing strings, one per generated sentence
    """
    # Generate sentences one by one and store
    sentences = [generate_sentence() for _ in range(n)]
    return sentences

In [151]:
sentences = generate(50)
print("Sentences:")
for sentence in sentences:
  print(sentence)

Sentences:
<s> <s> doesn't matter </s> </s>
<s> <s> i don't care if it's no chinese it will be going on thursday or friday or a weekend night </s> </s>
<s> <s> i would like to go for breakfast </s> </s>
<s> <s> i would like to go for lunch </s> </s>
<s> <s> i'm looking for italian restaurant for brunch </s> </s>
<s> <s> the distance is about half a mile </s> </s>
<s> <s> after ten p__m </s> </s>
<s> <s> uh find a japanese restaurant um casual for the evening would be good food i'm not very hungry i intend to spend uh between ten to fifteen dollar </s> </s>
<s> <s> hamburger sounds nice </s> </s>
<s> <s> start over please </s> </s>
<s> <s> any day </s> </s>
<s> <s> how about yangtze river </s> </s>
<s> <s> i wanna keep it under fifteen dollars </s> </s>
<s> <s> where do i find a really good greek restaurant something not as expensive </s> </s>
<s> <s> tell me more information about soup kitchen heike </s> </s>
<s> <s> expensive </s> </s>
<s> <s> no more than thirty dollars </s> </s>
<s>

### TASK - 4  = 5 points :
Measures the perplexity for the test sequence with your trained model. 
you may assume that this sequence may consist of many sentences "glued together"

The perplexity of the given sequence is the inverse probability of the test set, normalized by the number of words.


In [156]:
# Since this sequence will cross many sentence boundaries, we need to include 
# the begin- and end-sentence markers <s> and </s> in the probability computation. 
# We also need to include the end-of-sentence marker </s> 
# but not the beginning-of-sentence marker <s>) in the total count of word tokens N

def perplexity(test_sequence):
    """.
    Parameters:
      test_sequence (string): a sequence of space-separated tokens to measure the perplexity of

    Returns:
      float: the perplexity of the given sequence
    """ 

    # Replace out of vocab words with <UNK>, already done in score function
    # test_sequence = [token if token in vocab else UNK for token in test_sequence.split()]
    test_sequence = [token if token in vocab else UNK for token in test_sequence.split()]
    n_gram_tokens = make_ngrams(test_sequence, n_gram)
    n_minus_1_gram_tokens = make_ngrams(test_sequence, n_gram - 1)

    V = len(vocab)
    n_gram_total_occurence = sum(n_gram_counts.values())
    n_minus_1_gram_total_occurence = sum(n_minus_1_gram_counts.values())
    log_prob = 0.
    for i in range(len(n_gram_tokens)):
      log_prob += get_gram_log_prob(n_gram_counts, n_gram_tokens[i], n_gram_total_occurence, V) - get_gram_log_prob(n_minus_1_gram_counts, n_minus_1_gram_tokens[i+1], n_minus_1_gram_total_occurence, V)
    log_prob = log_prob / float(len(n_gram_tokens))
    perplexity = np.power(2, -log_prob)
    # Remove sentence begin markers from data for computing N
    # Get the probability for the sequence
    
    return perplexity

In [157]:
print(perplexity(" ".join(sentences[0:10])))

2.0169398427882417


### **Theory: (5 points)**
* Experiment n_gram model for n = [1,2,3..7] of your choice. Explain the best choice of n that generates more meaninful sentences.


# <CENTER> PART-B</CENTER> 

### OBJECTIVE : In this unsupervised learning task we are going to cluster wikipedia articles into groups using Hierarchical clustering

# **TASK-1 : 5 Points**
##Download articles from Wikipedia
In this section we will download articles from wikipedia and then cluster them into groups in the next step. You can select somewhat related topics or fetch the articles randomly. 
(Use dir() and help() functions or refer wikipedia documentation)
You may also pick any other data source of your choice instead of wikipedia.

In [ ]:
import wikipedia
from wikipedia.exceptions import WikipediaException
import pandas as pd

'''
 Generate a list of wikipedia article titles to cluster 
 You can maintain a static list of titles or generate them randomly using wikipedia library
 Some topics include:
 ["Northeastern Unversity", "Natural language processing", "Machine learning", "Quantum machine learning", "Artificial intelligence", "Data science", "Master in Data Science", 
 "Bank of America", "Visa Inc.", "European Central Bank", "Bank", "Financial technology","International Monetary Fund", 
 "Basketball", "Swimming", "Tennis", "Football", "College Football", "Association Football"]

 You can add more topics from different categories so that we have a diverse datset to work with. 
 Ex- About 10+ categories with 3+ article in each category
'''
# list of articles to be downloaded
topics = ["Northeastern Unversity", "Natural language processing", "Machine learning", "Quantum machine learning", "Artificial intelligence", "Data science", "Master in Data Science", 
 "Bank of America", "Visa Inc.", "European Central Bank", "Bank", "Financial technology","International Monetary Fund", 
 "Basketball", "Swimming", "Tennis", "Football", "College Football", "Association Football"]

# download and store all the articles in this variable
data=[]
for topic in topics[1:2]:
    titles = wikipedia.search(topic)
    print(titles)
    for title in titles:
        try:
            data.append(wikipedia.page(title).content)
        except WikipediaException:
            print(f"error when searching for {topic}")


# **TASK-2 : 5 Points**
#Cleaning the Data
In this step you will decide whether to clean the data or not. If you choose to clean, you may utilize the clean function from assignment 1

**Answer(1-3 sentences):** Why are you (or not) choosing to clean the data? Think in terms of whether cleaning the data will help in the clustering or not.

In [ ]:
# You can use Assignment 1's clean message function

# **TASK-3 : 10 Points**
##Vectorize the articles(5 points)
In this step, we will vectorize the text data to use in hierarchical clustering. You can use countVectorizer() or TfidfVectorizer() from sklearn library.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

##Plot Dendogram (5 points)
Now we will try to see the hierarchical relationship between articles using dendrogram.

In [ ]:
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as shc

After plotting the dendogram, you will see that if you cut the dendogram horizontally, you can seperate the data into groups. You will get different number of clusters depending on where you cut.

# **TASK-4 : 5 Points**
#Apply Clustering
In this step, we will assign cluster lables to each document/group using Agglomerative Hierarchical clustering.
We can decide number of clusters based on the dendogram and our requirement (how many categories we want).(eg. n_clusters = 3) 

In [ ]:
from sklearn.cluster import AgglomerativeClustering

# **TASK-5 : 5 Points**
#Word Clouds
Now, we will try to visualize top 50 words in each cluster using word clouds

In [ ]:
from wordcloud import WordCloud

for k in range(0, num_clusters):

Comment about the categorizion done by Hierarchical clustering. Are the groups meaningful?

# **TASK-6 : 10 Points**
# Apply Hierarchical clustering on spam dataset
Now we will apply Hierarchical clustering(HC) on a subset(modify the fraction argument of the sample() function) of Assignment 1 data. We will try to see if Hierarchical clustering can perform good or not for a supervised problem. We will follow the same steps as above and apply HC on the message column of spam.csv and categorize them into two clusters.

In [ ]:
from sklearn.utils import resample
# Read the data as done in Assignment 1
## Reading the data and removing columns that are not important. 
## Renaming the columns so that we understand the columns easily.

#Task 7 Conclusion(5 points)
Did Hierarchical clustering work as intended for spam classification? Why? 
(3-5 sentences)